In [4]:
import sys
from path_info import PROJECT_DIR
sys.path.append(PROJECT_DIR)

import torch

from src.utils_benchmark_functions_2 import test_function
from src.utils_benchmark_functions import discretize_function
from src.botorch_wrapper import DiscreteBO 

In [5]:
import torch
import torch.nn as nn
from torch.distributions.normal import Normal
from botorch.models.model import Model
from botorch.acquisition import UpperConfidenceBound
from botorch.optim import optimize_acqf
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.distributions import MultivariateNormal


class BayesianLinearRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(BayesianLinearRegression, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        # Parameters for the prior distributions of weights and biases
        self.w_mu = nn.Parameter(torch.zeros(input_dim, output_dim))
        self.w_log_sigma = nn.Parameter(torch.zeros(input_dim, output_dim))
        self.b_mu = nn.Parameter(torch.zeros(output_dim))
        self.b_log_sigma = nn.Parameter(torch.zeros(output_dim))
    
    def forward(self, x):
        w_sigma = torch.exp(self.w_log_sigma)
        b_sigma = torch.exp(self.b_log_sigma)
        
        # Sampling weights and biases
        w = self.w_mu + w_sigma * torch.randn_like(self.w_mu)
        b = self.b_mu + b_sigma * torch.randn_like(self.b_mu)
        
        return torch.matmul(x, w) + b
    
    def predict_dist(self, x):
        y = self.forward(x)
        
        # Calculating the uncertainty of the output
        w_sigma = torch.exp(self.w_log_sigma)
        b_sigma = torch.exp(self.b_log_sigma)
        
        # Calculating the standard deviation considering the uncertainty of weights and biases
        output_sigma = torch.sqrt(torch.matmul(x**2, w_sigma**2) + b_sigma**2)
        
        return Normal(y, output_sigma)


class BayesianMLP(nn.Module):
    def __init__(self, input_dim, min_val=None, max_val=None):
        super(BayesianMLP, self).__init__()
        self.hidden1 = nn.Linear(input_dim, 64)
        self.hidden2 = nn.Linear(64, 64)
        self.hidden3 = nn.Linear(64, 64)
        self.relu = nn.ReLU()
        self.bayesian_output = BayesianLinearRegression(64, 1)
        self.min_val = min_val
        self.max_val = max_val
    
    def forward(self, x):
        x = self.relu(self.hidden1(x))
        x = self.relu(self.hidden2(x))
        x = self.relu(self.hidden3(x))
        
        # Get output from Bayesian linear regression
        y_dist = self.bayesian_output.predict_dist(x)

        if self.min_val is not None or self.max_val is not None:
            # Clamp the output
            y_mean = torch.clamp(y_dist.mean, min=self.min_val, max=self.max_val)
        else:
            y_mean = y_dist.mean

        y_stddev = y_dist.stddev  # Keep the standard deviation as it is
        
        # Return new distribution
        return Normal(y_mean, y_stddev)


class BayesianMLPModel(Model):
    def __init__(self, input_dim, min_val=None, max_val=None):
        super().__init__()
        self.bayesian_mlp = BayesianMLP(input_dim, min_val, max_val)
        self.likelihood = GaussianLikelihood()
        self._num_outputs = 1

    def forward(self, x):
        return self.bayesian_mlp(x)
    
    def posterior(self, X, observation_noise=False, **kwargs):
        pred_dist = self.bayesian_mlp(X)
        mean = pred_dist.mean.squeeze(-1)  # Ensure mean is 2D
        stddev = pred_dist.stddev.squeeze(-1)  # Ensure stddev is 2D
        covar = torch.diag_embed(stddev**2)
        return MultivariateNormal(mean, covar)
    
    @property
    def num_outputs(self):
        return self._num_outputs
    
    @property
    def train_inputs(self):
        return self._train_inputs

    @property
    def train_targets(self):
        return self._train_targets

    def set_train_data(self, inputs=None, targets=None, strict=True):
        self._train_inputs = inputs
        self._train_targets = targets


def train_model(model, train_X, train_Y, num_epochs=1000, learning_rate=0.01):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    model.train()
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        output_dist = model(train_X)
        loss = -output_dist.log_prob(train_Y).mean()  # Minimize negative log likelihood
        loss.backward()
        optimizer.step()
    return model


In [3]:
# Define the test function
def test_function(x):
    if 0 <= x <= 1:
        return 10 + 10 * (x - 0.5)**2  # High penalty within [0, 1]
    else:
        return (x - 2)**2  # Quadratic function with a minimum at x = 2


# Convert test_function to a PyTorch tensor
def objective(x):
    return torch.tensor([test_function(xi.item()) for xi in x], dtype=torch.float32)


# Create initial data points
train_X = torch.rand(10, 1) * 4  # Generate 10 random initial points in the range [0, 4)
train_Y = torch.tensor([test_function(xi.item()) for xi in train_X], dtype=torch.float32).unsqueeze(-1)

print(f'train_X: {train_X}')
print(f'train_Y: {train_Y}')

train_X: tensor([[2.4328],
        [3.6670],
        [3.3378],
        [1.9746],
        [3.2818],
        [3.1092],
        [3.1636],
        [2.4338],
        [3.5061],
        [1.3407]])
train_Y: tensor([[1.8732e-01],
        [2.7788e+00],
        [1.7897e+00],
        [6.4470e-04],
        [1.6429e+00],
        [1.2303e+00],
        [1.3539e+00],
        [1.8821e-01],
        [2.2683e+00],
        [4.3472e-01]])


In [8]:
class DiscreteBO():
    def __init__(self, bounds, beta=2.0, beta_h=10.0, l_h=2.0):
        super().__init__(bounds)
        self.beta = beta
        self.beta_h = beta_h
        self.l_h = l_h
        self.l = 1.0

    def initialize(self, X_init, Y_init):
        self.train_X = X_init
        self.train_Y = Y_init
        self.model = self._fit_model()

    # def _fit_model(self):
    #     model = SingleTaskGP(self.train_X, self.train_Y)
    #     mll = ExactMarginalLogLikelihood(model.likelihood, model)
    #     fit_gpytorch_mll(mll)
    #     return model

    def _fit_model(self):
        # Define and fit the model
        model = BayesianMLPModel(input_dim=1)
        model.set_train_data(self.train_X, self.train_Y)
        model = train_model(model, self.train_X, self.train_Y)
        return model

    def acquisition_function(self, beta):
        return UpperConfidenceBound(self.model, beta=beta)

    def optimize_acquisition(self, acq_function):
        candidates, _ = optimize_acqf(
            acq_function,
            bounds=self.bounds,
            q=1,
            num_restarts=10,
            raw_samples=20,
        )
        return candidates

    def step(self):
        acq_function = self.acquisition_function(self.beta)
        new_X = self.optimize_acquisition(acq_function)

        new_X = torch.round(new_X)

        # Check if new_X is in train_X and adjust beta and l if necessary
        while (new_X == self.train_X).all(dim=1).any():
            self.adjust_beta_and_l()
            acq_function = self.acquisition_function(self.beta)
            new_X = self.optimize_acquisition(acq_function)
            new_X = torch.round(new_X)

        return new_X

    def update(self, new_X, new_Y):
        self.train_X = torch.cat([self.train_X, new_X], dim=0)
        self.train_Y = torch.cat([self.train_Y, new_Y], dim=0)
        self.model = self._fit_model()

    def adjust_beta_and_l(self):
        # Define the optimization problem to adjust beta and l
        def objective(params):
            delta_beta, l = params
            adjusted_beta = self.beta + delta_beta
            acq_function = self.acquisition_function(adjusted_beta)
            new_X = self.optimize_acquisition(acq_function)
            rounded_new_X = torch.round(new_X)

            penalty = float('inf')
            if (rounded_new_X == self.train_X).all(dim=1).any():
                penalty = 1000  # Arbitrary high value to avoid repetition

            return delta_beta + torch.norm(new_X - rounded_new_X).item() + penalty

        # Initial guess and bounds for delta_beta and l
        initial_guess = torch.tensor([0.0, self.l])
        bounds = [(0.0, self.beta_h), (1e-3, self.l_h)]

        # Optimize the objective function
        result = minimize(objective, initial_guess, bounds=bounds, method='L-BFGS-B')
        delta_beta, l = result.x
        self.beta += delta_beta
        self.l = l


In [6]:
# Optimization loop
for i in range(100):
    # Define and fit the model
    model = BayesianMLPModel(input_dim=1)
    model.set_train_data(train_X, train_Y)
    model = train_model(model, train_X, train_Y)

    # Set the model to evaluation mode
    model.eval()
    
    # Define the acquisition function
    acq_func = UpperConfidenceBound(model, beta=0.1)
    
    # Perform optimization
    bounds = torch.tensor([[0.0], [4.0]])
    candidate, acq_value = optimize_acqf(
        acq_function=acq_func,
        bounds=bounds,
        q=1,
        num_restarts=5,
        raw_samples=20,
    )
    
    print(f"Iteration {i+1}, Optimal candidate: {candidate.item()}")
    
    # Obtain new data point
    new_x = candidate
    new_y = torch.tensor([test_function(new_x.item())], dtype=torch.float32).unsqueeze(-1)
    
    # Update data
    train_X = torch.cat([train_X, new_x])
    train_Y = torch.cat([train_Y, new_y])


print()
print()
print()
print("hellooooo")
print(train_X[train_Y.argmin().item()])


Iteration 1, Optimal candidate: 3.809638023376465
Iteration 2, Optimal candidate: 0.2322077453136444
Iteration 3, Optimal candidate: 4.0
Iteration 4, Optimal candidate: 4.0
Iteration 5, Optimal candidate: 3.9056220054626465
Iteration 6, Optimal candidate: 2.9983606338500977
Iteration 7, Optimal candidate: 3.6666674613952637
Iteration 8, Optimal candidate: 0.0
Iteration 9, Optimal candidate: 4.0
Iteration 10, Optimal candidate: 1.810307264328003
Iteration 11, Optimal candidate: 1.6404427289962769
Iteration 12, Optimal candidate: 3.8088953495025635
Iteration 13, Optimal candidate: 0.9856517314910889
Iteration 14, Optimal candidate: 1.440605878829956
Iteration 15, Optimal candidate: 3.4827260971069336
Iteration 16, Optimal candidate: 2.7431647777557373
Iteration 17, Optimal candidate: 0.19765116274356842
Iteration 18, Optimal candidate: 3.5888752937316895
Iteration 19, Optimal candidate: 2.6874797344207764
Iteration 20, Optimal candidate: 1.287191390991211
Iteration 21, Optimal candidate:

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import sys
from path_info import PROJECT_DIR
sys.path.append(PROJECT_DIR)

from src.bnn import BayesianMLP

In [ ]:
# データのURL
url = "https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv"

# データの読み込み
df = pd.read_csv(url)

# 特徴量とターゲットの分割
X = df.drop(columns=['medv']).values
y = df['medv'].values

# データの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 特徴量の標準化
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# PyTorchテンソルへの変換
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [ ]:
# モデルの定義とオプティマイザの設定
model = BayesianMLP(input_dim=X_train.shape[1], min_val=0)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# トレーニング
model.train()
num_epochs = 10000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    output_dist = model(X_train)
    loss = -output_dist.log_prob(y_train).mean()  # 負の対数尤度を最小化
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# トレーニングデータでの予測
model.eval()
with torch.no_grad():
    train_output_dist = model(X_train)
    train_y_pred = train_output_dist.mean

# テストデータでの予測
with torch.no_grad():
    test_output_dist = model(X_test)
    test_y_pred = test_output_dist.mean

# 予測誤差の計算
mse = nn.MSELoss()
train_mse = mse(train_y_pred, y_train).item()
test_mse = mse(test_y_pred, y_test).item()

print('\n-------------------------------------------------')
print(f'Training Mean Squared Error: {train_mse:.4f}')
print(f'Test Mean Squared Error: {test_mse:.4f}')

RuntimeError: torch.clamp: At least one of 'min' or 'max' must not be None